<h1><center>Title Generation </center></h1>
<h2><center>Sequence-to-Sequence Text Summarization for Academic Journal Articles </center></h2>
<center>Karina Huang, Abhimanyu Vasishth, Phoebe Wong </center>
<center>AC209b: Advanced Topics in Data Science </center>
<center>Spring 2019 </center>

---

In [1]:
#import package dependencies
import re
import glove
import numpy as np
import pandas as pd
from collections import Counter
from sklearn.model_selection import train_test_split

from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

## 1. Introduction

In [ ]:
#abhi

## 2. Baseline Model - Nearest Neighbors with TF-IDF

In [ ]:
#abhi

---

## 3. Data Preprocessing for Recurrent Neural Network (RNN)

### 3.1 Data Cleaning
The original [NIPS dataset](https://www.kaggle.com/benhamner/nips-2015-papers/version/2) acquired from Kaggle included XXXX journal articles, most of which were missing abstracts. Our first attempt to clean the data was to find and extract abstracts for articles missing the piece of information. The `getAbstract` code performs the search for abstract in two steps. These two steps were results of ad-hoc identification of abstract extractions and recovered 3,250 abstracts. We removed all articles missing abstracts and formatted the text for data cleaning. Due to computational constraints, we subsetted articles with abstract of length 250 in words for the current study. The final dataset used included 4,638 observations without missing data in title or abstract. 

In [2]:
def formatText(x):
    "render space in text and text to lowercase"
    for i in range(len(x)):
        #check for data type
        if type(x[i]) == str:
            try:
                x[i] = x[i].replace('\n', ' ').lower()
            except:
                x[i] = x[i].lower()
    return x

def getAbstract(paper_text, methods = 1):
    "extract abstract from text in two steps"
    #step 1:
    #find 'abstract' in text
    #find the next word/phrase in all cap, wrapped in '\n'
    #extract everything in between as abstract
    if methods == 1:
        try:
            #find abstract
            a1 = re.search('abstract\n', paper_text, re.IGNORECASE)
            paper_text = paper_text[a1.end():]
            #find the next section in all cap
            a2 = re.search(r'\n+[A-Z\s]+\n', paper_text)
            return paper_text[: a2.start()]
        except:
            return np.nan
    #step 2:
    #find abstract in text
    #find next item wrapped between '\n\n' and '\n\n'
    #extract everything in between as abstract
    if methods == 2:
        try:
            a1 = re.search('abstract\n', paper_text, re.IGNORECASE)
            paper_text = paper_text[a1.end():]
            #find the next section in all cap
            a2 = re.search(r'\n\n+.+\n\n', paper_text)
            return paper_text[: a2.start()]
        except:
            return np.nan


def preprocessing(papers, formatCols = ['title', 'abstract','paper_text'], dropnan = False):
    "preliminary data preprocessing for model fitting"
    #avoid modifying original dataset
    papersNew = papers.copy()
    #replace missing values with nan
    papersNew.abstract = papersNew.abstract.apply(lambda x: np.nan if x == 'Abstract Missing' else x)
    #extract missing abstract in two steps
    #steps identified by ad-hoc examination of missing values
    for m in [1, 2]:
        #try searching for abstract in text if value is missing
        papersNew['abstract_new'] = papersNew.paper_text.apply(lambda x: getAbstract(x, methods = m))
        #replace nan in abstract with extracted abstract
        papersNew.loc[papersNew.abstract.isnull(), 'abstract'] = papersNew.abstract_new
        papersNew.drop(['abstract_new'], axis = 1, inplace = True)
    #format columns of interest
    papersNew[formatCols] = papersNew[formatCols].apply(lambda x: formatText(x), axis = 1)
    if dropnan:
        #drop na in abstract
        papersNew = papersNew.dropna(subset = ['abstract'])
        #append abstract and title length to data frame
        papersNew ['aLen'] = papersNew.abstract.apply(lambda x: len(x.split(' ')))
        papersNew ['tLen'] = papersNew.title.apply(lambda x: len(x.split(' ')))
    return papersNew

In [3]:
#load data
data = pd.read_csv('../data/papers.csv')
#preprocessing
dataNew = preprocessing(data, dropnan = True)
#subset articles with a length less than or equal to 250
data250 = dataNew[dataNew.aLen <= 250]
print('Final Dataset Used')
print('==================')
print('Number of observations: ', data250.shape[0])
print('Maximum title length: ', data250.tLen.max())

Final Dataset Used
Number of observations:  4638
Maximum title length:  20


### 3.1 Data Preprocessing for Model Training

After cleaning the data, we processed the titles and abstracts using `processText` below. The class object records and returns:

* number of unique words
* maximum sequence length (should be 250 as titles are shorter than abstracts)
* dictionaries for tokenization
* tokenized vector of titles and abstracts

Note that an important step of our tokenization was the qualification of rare, or unwanted, words. This is because NIPS articles are often written in laTex, and included scientific equations that may compromise the learning of important words. We approximated patterns of unwanted words and replaced them with an `<ign>` tag in the tokenization process. This resulted in 32,468 unique words in our dataset.

In [4]:
def qualify(word):
    '''helper function to select words for tokenization.'''
    #symbols
    symbols = """/?~`!@#$%^&*()_-+=|\{}[];<>"'.,:"""
    #abbreviations
    abb = """e.g.,i.e.,etal.,"""
    #disqualify empty space and words starting with symbol
    if len(word) < 1 or word[0] in symbols:
        return False
    elif len(word) > 2:
        #disqualify abbreviations
        if word in abb:
            return False
        #otherwise count all combinations with length > 2
        else:
            return True
    #if input length is one
    #count only if it is 'a'
    elif len(word) == 1:
        if word in ['a', 'i']:
            return True
    #with input length of 2
    #disqualify those with a symbol as the second character
    elif len(word) == 2:
        if word[1] not in symbols:
            return True
    #otherwise disqualify input
    else:
        return False

class processText:
    '''
    class object for data processing preperation for embedding training.

    Parameters:
    ===========
    1) textVec: list of array-like, vector of text in strings

    Methods:
    ===========
    1) updateMaxLen: count and update maximum sequence length
    2) getDictionary: update dictionaries of words and tokens,
        function called in `tokenize`
    3) tokenize: return tokenized vector of text for model training
    '''
    def __init__(self, textVec):

        #initiate class object
        self.textVec = list()
        for vec in textVec:
            #string to list
            vec = [x.strip().split(' ') for x in vec]
            self.textVec.append(vec)

        #prep  dictionaries for update
        self.word2idx = dict()
        self.idx2word = dict()
        self.maxLen = 0
        self.nUnique = 0

    def updateMaxLen(self):
        for vec in self.textVec:
            for txt in vec:
                #get length of sequence
                cntLen = len(txt)
                #update maximum sequence length
                if self.maxLen < cntLen:
                    self.maxLen = cntLen

    def getDictionary(self):

        if len(self.word2idx) != 0:
            print("Dictionary already updated.")

        else:
            #initiate dictionary updates
            #pad with 0
            #end of sequence as 1
            #ignored/disqualified words as 2
            #start tokenization at 3
            pad = 0
            eos = 1
            ign = 2
            start = 3

            self.word2idx['_'] = pad
            self.word2idx['*'] = eos
            self.word2idx['<ign>'] = ign

            for vec in self.textVec:
                for txt in vec:
                    for w in txt:
                        if qualify(w) == True:
                            if w not in self.word2idx.keys():
                                self.word2idx.update({w: start})
                                start += 1

            #update number of unique words in data set
            self.nUnique = start - 3
            #update idx to word dictionary
            self.idx2word = dict((idx,word) for word,idx in self.word2idx.items())

    def tokenize(self):
        #get dictionaries if function hasn't been called
        if len(self.word2idx) == 0:
            self.getDictionary()
        #cache list for tokenization
        tokenizedVec = list()
        for i in range(len(self.textVec)):
            vec = self.textVec[i]
            #cache list for the tokenized vector
            tempVec = list()
            for txt in vec:
                #cache list for sequence
                sVec = list()
                for w in txt:
                    #if word is in dictionary, tokenize
                    if w in self.word2idx:
                        sVec.append(self.word2idx[w])
                    #if word not in dictionary, tag as ignored
                    else:
                        sVec.append(self.word2idx['<ign>'])
                tempVec.append(sVec)
            tokenizedVec.append(tempVec)

        return tokenizedVec

In [5]:
#tokenize data
prep = processText(data250[['title', 'abstract']].values.T)
#update sequence length
prep.updateMaxLen()
#get dictionaries of word and tags
prep.getDictionary()
word2idx = prep.word2idx
idx2word = prep.idx2word

print('Number of unique words: ', prep.nUnique)
print('Maxmimum sequence length: ', prep.maxLen)
print('='*110)

#get tokenized vector of text
txtTokenized = prep.tokenize()
titles = txtTokenized[0]
abstracts = txtTokenized[1]
print('Example of tokenized title:\n {0} => {1}'.format(titles[0], [prep.idx2word[i] for i in titles[0]]))
print('='*110)
print('Example of tokenized abstract:\n {0} => {1}'.format(abstracts[0],[prep.idx2word[i] for i in abstracts[0]]))

Number of unique words:  32468
Maxmimum sequence length:  250
Example of tokenized title:
 [3, 4, 5, 6, 7, 8, 9] => ['self-organization', 'of', 'associative', 'database', 'and', 'its', 'applications']
Example of tokenized abstract:
 [42, 466, 64, 4, 580, 5, 5497, 431, 5498, 5499, 51, 9, 19, 321, 5500, 5501, 58, 5498, 5497, 176, 5502, 3251, 503, 51, 309, 5503, 75, 58, 619, 5504, 1743, 4, 5505, 42, 61, 4, 3, 431, 5506, 368, 42, 1019, 4, 5507, 1727, 5508, 10, 289, 4072, 4, 21, 5509, 75, 58, 5510, 5504, 5511, 42, 5512, 19, 187, 1181, 92, 7, 122, 19, 42, 319, 320, 321, 159, 1391, 5513] => ['an', 'efficient', 'method', 'of', 'self-organizing', 'associative', 'databases', 'is', 'proposed', 'together', 'with', 'applications', 'to', 'robot', 'eyesight', 'systems.', 'the', 'proposed', 'databases', 'can', 'associate', 'any', 'input', 'with', 'some', 'output.', 'in', 'the', 'first', 'half', 'part', 'of', 'discussion,', 'an', 'algorithm', 'of', 'self-organization', 'is', 'proposed.', 'from', 'an', 

Finally, we split our dataset into training ($\approx$72\%), validation ($\approx$8\%) and test set ($\approx$20\%). For consistency, we set the random state to 209.

In [6]:
#split data into train, validation, and test set
trainX, testX, trainY, testY = train_test_split(abstracts, titles, test_size = 0.2 , random_state = 209)
trainX, valX, trainY, valY = train_test_split(trainX, trainY, test_size = 0.1 , random_state = 209)

print('Number of training samples: ', len(trainX))
print('Number of validation samples: ', len(valX))
print('Number of test samples: ', len(testX))

Number of training samples:  3339
Number of validation samples:  371
Number of test samples:  928


## 4. Word Embeddings

In [ ]:
#embedding karina
#word2vec phoebe
#glove pre-trained abhi

### 4.3 Self-Trained GloVe Embeddings

One motivation to train our own embedding is that pre-trained word embeddings may not capture well the similarities and co-occurances between words in the current dataset. Because academic journal article come with many technical terms, it is possible that weights using pre-trained embeddings do not apply to these texts. As mentioned above, only half of the unique words in the current dataset were found in the pre-trained GloVe embeddings. Therefore, we experimented with training our own embedding matrix, in hopes that the initialized weights would help our models learn to summarize the abstracts more effectively. Note that the embedding matrix is only trained on the training examples split using the code in the data preprocessing section. This is because realistically we do not have access to the hold-out sets. Again, the trained embedding matrix dimension is (32,471, 100); the height corresponds to the sum of the number of unique words and the number of special tags (`<eos>`, `<ign>`, `<pad>`).

In [7]:
#get text for training
#remove ignored/disqualified words
#because we do not want to learn this tag
embedd_trainX = [[idx2word[x] for x in v if x != 2] for v in trainX]
embedd_trainY = [[idx2word[x] for x in v if x != 2] for v in trainY]
embeddTxt = trainX + trainY

#prep dictionary for embedding training
#drop pad, eos, and ignore tag
start = 0
embeddDict = dict()
for vec in embeddTxt:
    for w in vec:
        if w not in embeddDict.keys():
            embeddDict[w] = start
            start += 1

print('Number of unique words for embedding training: ', len(embeddDict))

Number of unique words for embedding training:  27141


In [8]:
#the code below were used for embedding training
#please see rnn_preprocessing for the execution history

# #train glove embedding 
# #creating a corpus object
# corpus_ = glove.Corpus(dictionary = embeddDict) 
# #training the corpus to generate the co occurence matrix which is used in GloVe
# corpus_.fit(embeddTxt, window = 10)
# #train embedding using corpus weight matrix created above 
# glove_ = glove.Glove(no_components = 100, learning_rate = 0.01, random_state = 209)
# glove_.fit(corpus_.matrix, epochs=50, no_threads=10, verbose = True)
# glove_.add_dictionary(corpus_.dictionary)

# #embedding matrix
# #initiate a matrix with shape 
# #(number of unique words in our dataset, latent dimension of embedding)
# embeddMatrix = np.zeros((len(word2idx), 100))
# #loop through trained embedding matrix to find weights of trained words
# for i, w in enumerate(word2idx):
#     try:
#         embeddVec = glove_.word_vectors[glove_.dictionary[w]]
#         embeddMatrix[i] = embeddVec
#     except:
#         continue

In [27]:
#load trained glove model
glove_ = glove.Glove.load('rnn_training_history/glove_.model')
#display 10 words most similar to 'stochastic' by glove training
print('Top 10 words most similar to "stochastic"')
print('=========================================')
for (i,j) in glove_.most_similar(word = 'stochastic', number = 10):
    print("word: {0} | cosine similarity: {1}".format(i, j))

Top 10 words most similar to "stochastic"
word: gradient | cosine similarity: 0.8991909690378893
word: three-composite | cosine similarity: 0.8937557315973716
word: descent | cosine similarity: 0.8744804215977376
word: proximal | cosine similarity: 0.76864338053953
word: optimization | cosine similarity: 0.7646848913256339
word: descent. | cosine similarity: 0.7640788665473602
word: accelerated | cosine similarity: 0.7629822276705301
word: gradients | cosine similarity: 0.7624208397216201
word: projected | cosine similarity: 0.7623820982466453


## 5. RNN Model 

In [ ]:
#karina

## 6. RNN Model with Attention and Context Mechanism

In [ ]:
#karina

## 7. Results

In [ ]:
#phoebe

## 8. Discussion

In [ ]:
#karina
#abhi
#PHOEBE